In [2]:
import pandas as pd
import numpy as np
import cufflinks as cf

from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

def parse_ts(ts):
    return pd.to_datetime(ts, unit="ms", utc=False)

headers = ['ts', 'X', 'Y', 'Z']
df = pd.read_csv('data/wash_idle.csv', names=headers, \
                 parse_dates=True,date_parser=parse_ts, index_col='ts')
df = df.drop(columns=["Z", "Y"])
iplot(df.iplot(asFigure=True, kind='scatter'))

In [6]:
len(df)

6901

In [44]:
import pandas as pd
import numpy as np
import cufflinks as cf

from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

LIMIT=.30

l1 = lambda x: x if x < LIMIT else LIMIT

def parse_ts(ts):
    return pd.to_datetime(ts, unit="ms", utc=False)

def parse_utc_ts(ts):
    d = pd.to_datetime(ts, unit="ms", utc=True)
    d = d.tz_convert('Asia/Kolkata')
    return d

def slice_stddev(df, batch_size, cols, aggregator=np.std):
    sddf = pd.DataFrame(columns=cols)
    sddf.set_index('ts', inplace=True)
    batches = int(len(df)/batch_size)
    for i in range(batches+1):
        start = i*batch_size
        end = (i+1)*batch_size
        slice = df[start:end] if end < len(df) else df[start:]
        if len(slice) < 1:
            continue
        ts = parse_ts(slice['ts'].values[0])
        sddf.loc[ts] = [
            l1(aggregator(slice['X'].values))
        ]
    return sddf

In [45]:
headers = ['ts', 'X', 'Y', 'Z']
df = pd.read_csv('data/wash_run1.csv', names=headers)
df = df.drop(columns=["Z", "Y"])
sdf = slice_stddev(df, 100, ['ts', 'X'], aggregator=np.std)
iplot(sdf.iplot(asFigure=True, kind='scatter'))

In [32]:
import plotly.graph_objects as go

idle_during_wash = sdf.loc['2019-12-13 06:07:00': '2019-12-13 07:45:43']['X']
idle_after_wash = sdf.loc['2019-12-13 08:57:00':]['X']
x_axis = sdf.loc['2019-12-13 06:05:57': '2019-12-13 07:45:43'].index
fig = go.Figure()
fig.add_trace(go.Scatter(y=[idle_during_wash.min() for i in range(len(x_axis))], 
                         x=x_axis, name='Washing(Min)', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=x_axis, y=idle_during_wash.values, name='Washing'))
fig.add_trace(go.Scatter(x=x_axis, y=idle_after_wash.values, name='Idle'))
fig.show()

In [41]:
headers = ['name','ts','X']
df = pd.read_csv('data/cotton_wash.csv', parse_dates=True,
                     date_parser=parse_utc_ts, index_col='ts', names=headers, header=None)
df = df.drop(columns=["name"])
iplot(df.iplot(asFigure=True, kind='scatter'))